# Battery Modelling 
[//]: \batterydrawing
[//]: \batterydiagram

## Aim: Model the patterns created by the electrodeposition in a battery.

- Electrodes: Cathode and Anode. 
- Electrolyte performs the reaction.
- Electrons produced through electrode material.

[//]: \cathodelab
[//]: \anodelab
[//]: \ratestable

# Partial Differential Equations

## Morphology

$$ \frac{\partial \eta}{\partial \tau}=
\underbrace{D_{s}^*\Delta \eta}_{\text{Diffusion}}+\underbrace{S_s^*}_{\text{Source}}
$$

[//]: \sourcemorph
$$ f(\eta,\theta)=- \eta\theta+\frac{\epsilon\eta^2}{1+\eta} $$ 

## Surface Chemistry

$$ \frac{\partial\theta}{\partial\tau}=\underbrace{D_{c}^*\Delta\theta}_{\text{Diffusion}}+\underbrace{S_{c}^*}_{\text{Source}}
$$
$$g(\eta,\theta)=A_0e^{a_0\eta+b_0\theta}\left(1-\eta\left(1+\frac{1}{\epsilon}\right)(1-\epsilon\eta)^2\right)$$

[//]: \sourcechem


# Weak Formulation

$$
\int \frac{\partial \eta}{\tau}\cdot \underbrace{\nu_1}_{\text{test function}}d x=\int \left(\nabla \eta \cdot \nabla \eta\underbrace{\nu_1}_{\text{test function}}+\rho f(\eta,\theta)\underbrace{\nu_1}_{\text{test function}}\right) \cdot d x
$$
$$
\int \frac{\partial\theta}{\partial\tau}\cdot \underbrace{\nu_2}_{\text{test function}} dx=\int \left(d \nabla \theta \cdot \nabla \underbrace{\nu_2}_{\text{test function}}+\sigma g (\eta,\theta) \underbrace{\nu_2}_{\text{test function}}\right)\cdot d x
$$


11# Numerical Model

## Time Discretisation

$$
\int \underbrace{\frac{\eta_{m+1}-\eta_{m-1}}{2\Delta\tau}}_{\substack{\text{Approximated} \\ \text{Derivative}\\ \text{at time} \\ \text{step $m$}}}\cdot \nu_1 d x
=\int \left(\nabla\eta_m \cdot \nabla \nu_1+\rho f(\eta_m,\theta_m)\nu_1\right) \cdot d x
$$
$$
\int \underbrace{\frac{\theta_{m+1}-\theta_{m-1}}{2\Delta\tau}}_{\substack{\text{Approximated} \\ \text{Derivative}\\ \text{at time} \\ \text{step $m$}}}\cdot \nu_2 d x
=\int \left(d \nabla \theta_m \cdot \nabla \nu_2 +\sigma g(\eta_m,\theta_m)\nu_2\right) \cdot dx
$$

The solution for time step $m+1$ is therefore given in terms of the solution at $m$ and $m-1$. We can rewrite this into the form $F(u_{m+1},v_i)=0$ where $u$ is either $\eta$ or $\theta$ and $i$ is $1$ or $2$ respectively. $F$ is nonlinear but can be split into the bilinear part $a(u,v)$ and the linear part $L(v)$.
$$
a_{\eta}(\eta_{m+1},\nu_1)=\int \eta_{m+1}\cdot \nu_1 \cdot d x
$$
$$
L_{\eta}(\nu_1)
=\int \left(\eta_{m-1}\nu_1+2\Delta \tau \nabla\eta_m \cdot \nabla \nu_1+2\Delta \tau \rho f(\eta_m,\theta_m)\nu_1\right) \cdot d x
$$
$$
a_{\theta}(\theta_{m+1},\nu_2)=\int\theta_{m+1}\nu_2 \cdot d x
$$
$$
L_{\theta}(\nu_2)
=\int \left(\theta_{m-1}+2\Delta \tau d \nabla \theta_m \cdot \nabla \nu_2 +2\Delta \tau \sigma g(\eta_m,\theta_m)\nu_2\right) \cdot dx
$$


# Download Packages after initial launch
The following code block only needs to be run when the notebook is first launched. You can then comment this out for future runs.

In [ ]:
import firedrake
from firedrake import mesh
from firedrake import functionspace as fs
from firedrake import Function

# Import Packages

In [ ]:
# Packages for loading data
import pandas as pd
import cv2
import os

In [ ]:
# Packages for managing data
import numpy as np
from itertools import product

In [ ]:
# Packages for running dolfinx 
import ufl
from mpi4py import MPI
from petsc4py import PETSc

In [ ]:
# Packages for plotting
import pyvista as pv
import matplotlib.pyplot as plt

## Input parameters

All input paramters should be entered into Inputs.csv

In [ ]:
def check_steps(meshpars,dt):
    '''Check that :math:'dt<h**2/2'
    :param meshpars: Array for the axis limits and number of steps on each axis [nx,ny,xm,ym,x0,y0]
    :param dt: time step, float
    return True if valid False if not'''
    nx,ny,xm,ym,x0,y0=meshpars
    h=min((xm-x0)/nx,(ym-y0)/ny)
    print(meshpars)
    #assert h<1
    check=bool(dt<h**2/2)
    return check

In [ ]:
def read_inputs(filename):
    '''Reads the inputs from csv. The input variables correspond to #FIXME-InsertModelType
    :param filename: filename where inputs are located includes path and file type
    * mesh parameters
    :rtype: [[],[]]
    :returns: [mesh parameters, PDE parameters]
    '''
    # Declare Global parameters
    global StSt
    global perm
    inputdata = pd.read_csv(filename)
    datadict=dict(zip(list(inputdata.label), list(inputdata.value)))
    # Dicretisation Parameters
    nx=datadict['nx']
    ny=datadict['ny']
    xm=datadict['xm']
    ym=datadict['ym']
    x0=datadict['x0']
    y0=datadict['y0']
    meshpars=[nx,ny,xm,ym,x0,y0]
    # PDE parameters
    StSt=[datadict['SteadyStateX'],datadict['SteadyStateY']]
    tol =datadict['tol']
    eps =datadict['epsilon']
    rho =datadict['rho']
    sig =datadict['sigma']
    a0  =datadict['a0']
    b0  =datadict['b0']
    A0  =datadict['A0']
    d   =datadict['d']
    dt  =datadict['dt']
    # Check that dt fits bound from h #TODO
    T   =datadict['T']
    perm=datadict['perm']
    pdes_pars=[StSt,tol,eps,rho,sig,a0,b0,A0,d,dt,T,perm]
    assert check_steps(meshpars,dt)
    return [meshpars,pdes_pars]

In [ ]:
def numfix(j):
    if j>=1:
        return '%04d'%j
    else: return '%04d'%(1000*j)

## Save and Load output variables
`save_var` takes the input variable, it's name and the folder to contain it in. If the desired directory does not exist it will be created. Default variable type is `.npy`.

In [ ]:
def check_folder(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    return

In [ ]:
def check_folder_nest(dirlist):
    initpath=''
    for d in dirlist:
        check_folder(initpath+d)
        initpath=initpath+d
    return

In [ ]:
def save_var(var,varname,foldername,typ='.npy'):
    ''' Saves the variable `var' with the name `varname' inside the folder `foldername'
    :param var: numpy variable
    :param varname: string for the variable name
    :param foldername: string indicating where the variable should be saved.
    :param typ: variable extension. Default to .npy
    :returns: nothing
    '''
    # Check the folder path exists and create it if not
    check_folder(foldername)
    # Save variable
    filename=foldername+'/'+varname+typ
    np.save(filename,var)
    return

In [ ]:
def load_file(dirname,fname,typ='.npy'):
    var=np.load(dirname+'/'+fname+typ)
    return var
def load_files(dirname,file_list,typ='.npy'):
    out=list()
    for fname in file_list:
        try: f=load_file(dirname,fname)
        except:
            if not os.path.exists(dirname): raise Error('directory does not exist, check path')
            else: raise Error('unable to load file')
        out.append(f) 
    return out

## Plots and Video
### Create a video using a sequence of plots.
The function `plot_animation` plots a heatmap for the variable values at each time step. These heatmaps are then combined to create a video animation. 

Currently the plots load the variables from the numpy arrays. The next step is to plot the dolfinx variables since this will translate better into different meshes.

# Update
Plotting functions have been improved in `PlottingFunctions.py`, import and use these functions instead

In [ ]:
def plot_animation(nt,varname='var',ResultsFolder=''):
    ''' Plotting sequence of heatmaps for a variable `var' and create animation.
    :param domain: numpy array containing the measurements for the envrionment [xmin,ymin,xmax,ymax]
    :param var: numpy array containing variable values
    :param tvec: numpy array containing the time values which correspond with the var results.
    :param varname: string corresponding to the name of var
    :param ResultsFolder: folder where the plots should be save. default ''
    '''
    #_,_,T=var.shape
    tp=1 # Time pause for video
    check_folder(ResultsFolder)
    filename=ResultsFolder+'/'+varname
    img_array=[]
    for j in range(50):
        if j%10==0:
            filesave=filename+numfix(j)+'.png'
            #tstring=varname+'(x,y), time {:.2f}'.format(tstep)
            #sns.heatmap(var[:,:,j],vmin=varmin,vmax=varmax).set(title=tstring)
            #im = plt.imshow(var[:,:,j],vmin=varmin,vmax=varmax)
            #im.title(tstring)
            #plt.savefig(filesave)
            #plt.close('all')
            img = cv2.imread(filesave)
            if isinstance(img,type(None)):
                pass
            else:
                height, width, layers = img.shape
                size = (width,height)
                img_array.append(img)
    out = cv2.VideoWriter(filename+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), tp, size)
    for j in range(len(img_array)):
        out.write(img_array[j])
    out.release()
    del img_array
    return


In [ ]:
def line_plot(f,t,varname,tname,ResultsFolder=''):
    ''' Plot f as a function of t. label x axis with `tname` 
    and y axis with `varname` save in ResultsFolder'''
    check_folder(ResultsFolder)
    if len(t)>len(f):
        t=t[-len(f)-1:-1]
    elif len(f)>len(t):
        f=f[-len(t)-1:-1]
    plt.figure()
    plt.plot(t,f)
    plt.xlabel(tname)
    plt.ylabel(varname)
    plt.savefig(ResultsFolder+'/'+varname+'_vs_'+tname+'.png')
    plt.close()
    return


In [ ]:
def plot_function(t, uh_cls,j=-1,ResultsFolder='/'):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    pv.set_jupyter_backend("ipygany")
    if j==-1:
        j=t
    p = pv.Plotter()
    if j<0:
        p.off_screen=False
    else: p.off_screen=True
    check_folder(ResultsFolder)
    uh=uh_cls.var
    filename=ResultsFolder+'/'+uh_cls.name
    V=uh_cls.function_space
    grid = pv.UnstructuredGrid(*plot.create_vtk_mesh(V))
    # Update point values on pyvista grid
    grid.point_data[f"u({t})"] = uh.x.array.real
    # Warp mesh by point values
    warped = grid.warp_by_scalar(f"u({t})", factor=1.5)
    filesave=filename+numfix(j)+'.png'
    tstring=uh_cls.name+'(x,y), time {:.2f}'.format(t)
    p.add_title(tstring)

    # Add mesh to plotter and visualize in notebook or save as figure
    actor = p.add_mesh(warped)
    if not p.off_screen:
        p.show()
    else:
        pv.start_xvfb()
        figure_as_array = p.screenshot(filesave)
        # Clear plotter for next plot
        p.remove_actor(actor)
    p.close()
    change_owner()
    print('Plot saved at',filesave)
    if DEBUG:
        print('Plots exists status',os.path.exists(filesave))
    return

In [ ]:
def plotting(xvar,varnames,ResultsFolder=''):
    '''Plots output variables from simulation and errors over time
    #FIXME-time plots need to be added
    :param var 3: numpy variable for x-axos(or similar dimensional variable)
    :param varnames: list of names corresponding to variables to plot
    :param ResultsFolder: string indicating the folder path where the plots should be save
    '''
    for vname in varnames:
        plot_animation(len(xvar),vname,ResultsFolder)
    return

## Source Functions

In [ ]:
def f_func(r_mesh,eta,theta,consts,test=0):
    _,eps,_,_,_,_,_,_,_=consts
    #Eps=fem.Constant(r_mesh, PETSc.ScalarType(eps))
    #return - eta*theta+eps*eta**2/(1+eta) 
    # Pure Diffusion (Heat Eqn)
    if test:
        return 0
    else:
        return - eta*theta+eps*eta**2/(1+eta) 
def g_func(r_mesh,eta,theta,consts,test=0):
    _,eps,_,_,a0,b0,A0,_,_=consts
    #Eps  =fem.Constant(r_mesh, PETSc.ScalarType(eps))
    #a0cst=fem.Constant(r_mesh, PETSc.ScalarType(a0))
    #b0cst=fem.Constant(r_mesh, PETSc.ScalarType(b0))
    #A0cst=fem.Constant(r_mesh, PETSc.ScalarType(A0))
    if test:
        return 0#fem.Constant(r_mesh, PETSc.ScalarType(0))
    else:
        return A0*ufl.exp(a0*eta+b0*theta)*(1-eta*(1+(1/eps)*((1-eps*eta)**2)))

## Linear and Bilinear Form


In [ ]:
def linear_form_func(varkey,nsteps,var_cls,othervar_cls,testvar,consts,test=0):
    '''The function `linear_form_func` defines the linear form for the variational problem. 
    By defining this outside the main simulation the same main program can be used for 
    different variational problems.
    :param varkey: either 'e' or 't' this gives which variable to take the form form.
    :param nsteps: is an integer giving the number of timesteps featuring in the linear form.
    :param var: `if nsteps=1:firedrake Function. 
                `if nsteps>1:
                    var=(var0,var1,...,var_{nsteps-1})
                    type(var_i)=`firedrake` Function`
    :param testvar: `dolfinx.ufl` Test Function
    :param consts: The constants required for the function declaration.
                   consts= [eps,rho,sig,a0,b0,A0,d,dt]
    :rtype: `ufl` expression
    :return: L=linear form
    '''
    _,_,rho,sig,_,_,_,d,dt=consts
    othervar=othervar_cls.var
    if nsteps==1:
        initvar=var_cls.var
        r_mesh=var_cls.mesh
    elif nsteps==2:
        var0_cls,var1_cls=var_cls
        var0=var0_cls.var
        var1=var1_cls.var
        r_mesh=var0_cls.mesh
    print('Heateqn test',test)
    if nsteps==1:
        divterm=initvar*testvar
        if varkey=='e':
            L=(dt*rho*f_func(r_mesh,initvar,othervar,consts,test)*testvar
             +divterm)*ufl.dx
        if varkey=='t':
            L=(dt*sig*g_func(r_mesh,initvar,othervar,consts,test)*testvar
             +divterm)*ufl.dx
    elif nsteps==2:
        divterm=var1*testvar
        if varkey=='e':
            L=(2*dt*rho*f_func(r_mesh,var0,othervar,consts,test)*testvar
             +divterm)*ufl.dx
        if varkey=='t':
            L=(2*dt*sig*g_func(r_mesh,var0,othervar,consts,test)*testvar
             +divterm)*ufl.dx
    return L
            


In [ ]:
def bilinear_form_func(varkey,tmpvar,testvar,consts,nsteps,r_mesh,test=0):
    ''' bilinear form for variational problem.
    :param varkey: either 'e' or 't' this gives which variable to take the form form.
    :param tmpvar: is a `dolfinx.ufl` TrialFunction :math:`v`
    :param testvar: is a `dolfinx.ufl` Test Function. :math:`\nu`
    :param consts: The constants required for the function declaration.
                   consts= [tol,eps,rho,sig,a0,b0,A0,d,dt]
    :param nsteps: is an integer giving the number of timesteps featuring in the linear form.
    :param mesh:   `firedrake` mesh
 
    
    ..math::
        if nstep==1
            `a(v,\nu)=\int d*dt*\nabla u \cdot \nabla \nu \cdot dx+\int v\nu\cdot d x`
        elif nstep==2
            `a(v,\nu)=\int 2*d*dt*\nabla u \cdot \nabla \nu \cdot dx+\int v\nu\cdot d x`
    
    :rtype: `ufl` expression
    :returns: a = bilinear form
    '''
    # bilinear form -trial function must go before test function
    _,_,_,_,_,_,_,d,dt=consts
    if varkey=='e':
      D=1.0
    elif varkey=='t':
      D=d
    if nsteps==1:
        a=D*dt*ufl.dot(ufl.grad(tmpvar),ufl.grad(testvar))*ufl.dx+tmpvar*testvar*ufl.dx
    elif nsteps==2:
        a=2*D*dt*ufl.dot(ufl.grad(tmpvar),ufl.grad(testvar))*ufl.dx+tmpvar*testvar*ufl.dx
    return a

The function `convert_vec_to_array` takes a `dolfinx.fem` Function in vector form then converts it into an array. This assumes that the mesh elements are equally spaced and the domain is rectangular. Once `xdmf` saving and plotting is worked out this function will become unecessary.

In [ ]:
def create_xdmf(u,MeshFolder=''):
    xdmf_file=io.XDMFFile(u.mesh.comm,MeshFolder+'/'+u.name+'.xdmf','w')
    xdmf_file.write_mesh(u.mesh)
    return xdmf_file

## Spatial Variable
class object for all variables which are functions of the space. The also allows us to set the name of a variable at creation and retrieve this name later without needing to maintain indexing positions between lists.

In [ ]:
class spatial_variable:
    def __init__(s,V,r_mesh,name=''):
        s.var=f.Function(V)
        s.name=name
        s.mesh=r_mesh
        s.function_space=V
        s.char_key='o' # Indictates other for when the character key is not set
        s.outfile=File(s.name+'.pvd')
    def set_name(s,namestr):
        s.name=namestr
        return
    def set_char_key(s,ck):
        s.char_key=ck
        return
    def get_name(s):
        return s.name
    def get_var(s):
        return s.var
    def get_mesh(s):
        return s.mesh
    def get_function_space(s):
        return s.function_space
    def var_vals(s):
        return s.var.vector.array[:]
    def print_var_vals(s):
        return str(s.var_vals())
    def __str__(s):
        rstr=' Spatial Variable '+s.name+'\n'
        rstr+=' Function Value '+s.print_var_vals()
        return rstr
    def update_var(s,cls_2):
        s.var.assign(cls_2.var)
        return
    def reset_var(s):
        s.var=fem.Function(s.function_space)
        return s.var
    def initialise(s,char='n'):
        #perm and StSt must be declared as global
        if char=='n':
            print('Character key not input so using class key',s.char_key)
            char=s.char_key
        if char=='e':
            s.var.interpolate(rand_scal_eta)
        elif char=='t':
            s.var.interpolate(rand_scal_theta)
        else:
            print('No initial func for character key')
            s.var.interpolate(initial_func)
        return s.var
    def store_tstep(s,t):
        s.outfile.write(s.var,time=t)
        return
        

## Generate Initial State
The initial conditions for both $\eta_0$ and $\theta_0$ is given by the steady state and a small random permuation. The interpolation function can't take in additional variables so the steady state and permuaton amplitude need to be defined globally.

In [ ]:
def rand_scal_eta(x):
    return StSt[0]+perm*(np.random.rand(x.shape[1])-np.random.rand(x.shape[1]))
def rand_scal_theta(x):
    return StSt[1]+perm*(np.random.rand(x.shape[1])-np.random.rand(x.shape[1]))
def Initial_eta_func(out):
    #perm and StSt must be declared as global
    out.interpolate(rand_scal_eta)
    return out
def Initial_theta_func(out):
    #perm and StSt must be declared as global
    out.interpolate(rand_scal_theta)
    return out
def initial_func(x, a=5):
    return np.exp(-a*(x[0]**2+x[1]**2))

## Boundary Conditions
Set zero flux boundary conditions. Neumann conditions are the default for the solver so are not set.


In [ ]:
# def boundary_condition(val,r_mesh,V):
#     # Create boundary condition
#     fdim = r_mesh.topology.dim - 1
#     boundary_facets = mesh.locate_entities_boundary(
#         r_mesh, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
#     bc = fem.dirichletbc(PETSc.ScalarType(val), fem.locate_dofs_topological(V, fdim, boundary_facets), V)
#     return bc

## Solvers
Using the bilinear and linear forms generate a solver for estimating the solution. The linear form vector is returned for input into the update function later.
Boundary conditions are also generated and returned for updating later.

In [ ]:
#def create_solver(a,L,r_mesh,V,bc):
#     # Set up the matrix and vector
#     bilinear_form = fem.form(a)
#     linear_form = fem.form(L)
#     A = fem.petsc.assemble_matrix(bilinear_form,bcs=[bc])
#     A.assemble()
#     b = fem.petsc.create_vector(linear_form)
#     # Create the solver
#     solver = PETSc.KSP().create(r_mesh.comm)
#     solver.setOperators(A)
#     solver.setType(PETSc.KSP.Type.PREONLY)
#     solver.getPC().setType(PETSc.PC.Type.LU)
#     return solver,b,bc

In [ ]:
# def UpdateSolver(b,a,L,bc):
#     bilinear_form = fem.form(a)
#     linear_form = fem.form(L)
#     with b.localForm() as loc_b:
#         loc_b.set(0)
#     fem.petsc.assemble_vector(b, linear_form)
#     # Apply Dirichlet boundary condition to the vector
#     fem.petsc.apply_lifting(b, [bilinear_form], [[bc]])
#     b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
#     fem.petsc.set_bc(b, [bc])
#     return b

par_string generates a suffix string indicating the parameter set for the simulation

In [ ]:
def par_string(dt,T,perm,hx,hy):
    '''Generate string for appending to folder name. '''
    return 'dt'+numfix(dt)+'_T'+numfix(T)+'_perm'+numfix(perm)+'_h'+numfix(min(hx,hy))

check_out returns True if any of the vector e or t are NaN

In [ ]:
def check_output(varlist):
    nanlist=list()
    for v in varlist:
        arr=np.array(v)
        arrsum=np.sum(arr)
        nanlist+=arrsum
    check=any([np.isnan(n) for n in nanlist])
    return check

check_zero is True if the vector v is all zeros. False if not

In [ ]:
def check_zero(v):
    return np.all((v==0))

Error Calculation

$e(v,w)=\frac{||v-w||_2}{||v||_2}$

In [ ]:
def norm_res(v,w):
    diff=np.linalg.norm(v-w)
    bot=np.linalg.norm(v)
    return diff/bot

# Main simulation

The majority of the work for the main program is done inside `Simulations`

In [ ]:
def Simulations(filename,MeshFolder,ResultsFolder,varnames,test=0,simsteps=1):
    ''' This function runs everything required to compute the simulations results.
    Note that this does not plot the results. The results are saved to numpy files which may be plotted later.
    :param filename: string containing the filename for the csv of input values. Includes path.
    :param MeshFolder: where to save the output files
    :param varnames: array of strings indicating variable names
    *Calls :py:func:'read_inputs(filename)' to load the inputs.
    *Computes mesh
    *Initialises the variables
    *Estimates initial conditions and time step 1.
    *Time stepping is a forwards Euler method.
    *Uses FEM implementation to estimate the variables for each time stepping.
    *Saves numpy arrays for the variables over discrete space and time.
    :returns: nothing computations are saved
    '''
    pars,pdes_pars=read_inputs(filename)
    etaname,thetaname,tname,domname,etaresname,theresname=varnames
    # nx,ny are number of elements on x and y axis
    # xm, ym are maximum x and y values
    # x0,y0 are minimum x and y values
    # hx, hy are meshwidth on the x and y axis.
    nx,ny,xm,ym,x0,y0=pars
    hx=(xm-x0)/nx
    hy=(ym-y0)/ny # mesh size
    boundaries=np.array([x0,y0,xm,ym])
    # Parameters for the PDEs
    StSt,tol,eps,rho,sig,a0,b0,A0,d,dt,T,perm=pdes_pars
    parstr=par_string(dt,T,perm,hx,hy)
    check_folder(ResultsFolder)
    check_folder(MeshFolder)
    MeshFolder=MeshFolder+'/'+parstr
    ResultsFolder=ResultsFolder+'/'+parstr
    check_folder(ResultsFolder)
    check_folder(MeshFolder)
    consts=pdes_pars[1:10] # These parameters are required for g_func, f_func and the variational form

    # Simulation parameters--------------------------------------------
    h = max(hx,hy)      # mesh size
    nt=int(T/dt)
    tvec = np.arange(0,T,dt)

    # Convert back to integer for array initialisation later.
    nx=int(nx)
    ny=int(ny)
    nt=int(nt)

    # Initialisation---------------------------------------------------
    # Regular python variables for outputs------------------------------
    #eta=np.zeros((nx+1,ny+1,nt))
    #the=np.zeros((nx+1,ny+1,nt))

    # FeNiCs variables--------------------------------------------------
    # Initialise the space ---------------------------------------------
    comm=MPI.COMM_WORLD
    bounds=(np.array([x0,y0]),np.array([xm,ym]))
    Ns=(nx,ny)
    r_mesh=mesh.UnitSquareMesh(Ns)
    V = fs.FunctionSpace(r_mesh,'Lagrange',1)
    # Function(V) indicates a scalar variable over the domain.
    # TestFunction(V) is a scalar test function over the domain for the weak problem.
    # TrialFunction(V) is the scalar function to be found in the solver.
    eta0_cls=spatial_variable(V,r_mesh,etaname)
    eta0_cls.set_char_key('e')
    the0_cls=spatial_variable(V,r_mesh,thetaname)
    the0_cls.set_char_key('t')
    #Initialise time step 0
    eta0_cls.initialise()
    the0_cls.initialise()
    eta0_cls.store_tstep(0)
    
    # Create xdmf file
    #eta_xdmf=create_xdmf(eta2_cls,MeshFolder)
    #the_xdmf=create_xdmf(the2_cls,MeshFolder)
    #eta_xdmf.write_function(eta2_cls.var,0)
    #the_xdmf.write_function(the2_cls.var,0)
    
    
    # Define one-step variational problem ---------------------------------------
    vars_cls=(eta0_cls,the0_cls)
    Lvec,avec=first_step(vars_cls,consts,test)
    Le,Lt=Lvec
    ae,at=avec
    Solver_e=NonlinearVariationalProblem(Le-ae,eta1_cls.var)
    Solver_t=NonlinearVariationalProblem(Lt-at,the1_cls.var)
    Solvers=(Solver_e,Solver_t)
    #DEBUG
    if False:
        pass
    #DEBUGEND
    if simstep==2:
        pass
    elif simstep==1:
        vars_cls, eNorm, tNorm=one_step_loop(vars_cls,Solvers,Lvec,avec,consts,tvec,ResultsFolder)
        eta2_cls,the2_cls=vars_cls
    os.rename(eta2_cls.name+'.pvd',MeshFolder+'/'+eta2_cls.name+'.pvd')
    os.rename(the2_cls.name+'.pvd',MeshFolder+'/'+the2_cls.name+'.pvd')
    save_var(tvec,tname,MeshFolder)
    save_var(boundaries,domname,MeshFolder)
    save_var(eNorm,etaresname,MeshFolder)
    save_var(tNorm,theresname,MeshFolder)
    print('Results saved in',MeshFolder)
    return eta2_cls,the2_cls,parstr
    

In [ ]:
def first_step(vars_cls,consts,test=1):
    '''Define variational problem ---------------------------------------  
    linear form -function must go before test function.
    :param vars_cls: list of all the variables as thier `spatial_variable` class object. :math:`var0_cls,var1_cls=vars_cls`
    :param testvars: tuple for the two test functions. :math:`v0,v1=testvars`
    :param trialvars: tuple of the trial functions. :math:`tmpv0,tmpv1=trialvars`
    :param consts: The constants required for the functions
    :param test: Indictator of test case. 
    - Heat Eqn test test=1
    - Full Model test=0 

    
    Creates linear form using `linear_form_func` and bilinear form using `bilinear_form_func`. 
    Creates the one step form for each variable for these.
    
    Uses these forms to create the solvers which are returned.
    
    - Solvers=(Solver_0,Solver_1)
    
    :returns: Solvers,bvecs
    '''
    var0_cls,var1_cls=vars_cls
    V                =var0_cls.function_space
    r_mesh           =var0_cls.mesh
    tmpv0   = ufl.TrialFunction(V)
    tmpv1   = ufl.TrialFunction(V)
    v0      = ufl.TestFunction(V)
    v1      = ufl.TestFunction(V)
    simsteps=1
    L0=linear_form_func(var0_cls.char_key,simsteps,var0_cls,var1_cls,v0,consts,test) # Linear form for eta with time step 1
    L1=linear_form_func(var1_cls.char_key,simsteps,var1_cls,var0_cls,v1,consts,test)
    a0=bilinear_form_func(var0_cls.char_key,tmpv0,v0,consts,simsteps,r_mesh,test=0)
    a1=bilinear_form_func(var1_cls.char_key,tmpv0,v0,consts,simsteps,r_mesh,test=0)
    
    # Solvers - Get time step 1
    #bc0=var0_cls.boundary_cond()
    #bc1=var1_cls.boundary_cond()
    #Solver_0,bvec0,bc0=create_solver(a0,L0,r_mesh,V,bc0)
    #Solver_1,bvec1,bc1=create_solver(a1,L1,r_mesh,V,bc1)
    #bvec0=UpdateSolver(bvec0,a0,L0,bc0)
    #bvec1=UpdateSolver(bvec1,a1,L1,bc1)
    #Solvers=(Solver_0,Solver_1)
    #bvecs=(bvec0,bvec1)
    #bcvec=(bc0,bc1)
    Lvec=(L0,L1)
    avec=(a0,a1)
    return Lvec,avec


In [ ]:
def one_step_loop(vars_cls,Solvers,Lvec,avec,consts,tvec,ResultsFolder=""):
    ''' Solve the spatial problem for each time step using the one step solver. 
    Solutions are plotted at each time step and the output is the variable with the values for the final time step along with the resiudal vector.
    '''
    var00_cls,var10_cls=vars_cls
    V                  =var00_cls.function_space
    r_mesh             =var00_cls.mesh
    var01_cls=spatial_variable(V,r_mesh,var00_cls.name+'1')
    var01_cls.set_char_key(var00_cls.char_key)
    var11_cls=spatial_variable(V,r_mesh,var10_cls.name+'1')
    var11_cls.set_char_key(var10_cls.char_key)
    L0,L1            =Lvec
    a0,a1            =avec
    Norm0=np.zeros(len(tvec))
    Norm1=np.zeros(len(tvec))
    Norm_t=1
    tol=consts[0]

    # TIME STEPPING COMPUTATIONS------------------------------------------------------

    # Begin Iterative Simulations --------------------------------------
    check_out=False
    dt=tvec[1]-tvec[0]
    Results0=[Function(var00_cls.var)]
    Results1=[Function(var00_cls.var)]
    Solver0,Solver1=Solvers
    for tstep,t in enumerate(tvec):
        if check_out: # When the residual is reached a steady state has been reached.
            print('Nans in output ')
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        elif Norm_t <=tol: # When the residual is reached a steady state has been reached.
            print('tolerance met %.6f'%Norm_t)
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        elif tstep+2>=len(tvec): 
            print('Time steps exceeded maximum')
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        else:
            #print('Solving for time step',t)
            # Update the right hand side reusing the initial vector
            Solver0.solve()
            Results0.append(Function(var01_cls.var))
            Solver1.solve()
            Results1.append(Function(var11_cls.var))
            # Write the solution to the xdmf file
            #eta_xdmf.write_function(eta2_cls.var, dt*(tstep+2))
            #the_xdmf.write_function(the2_cls.var,dt*(tstep+2))
            varlist=(var01_cls.var_vals(),var11_cls.var_vals())
            check_out=check_output(varlist)
        
            # Calculate the residual
            Norm0[tstep+1]=norm_res(var01_cls.var_vals(),var00_cls.var_vals()) # Sqrt of the sum of the sqs
            Norm1[tstep+1]=norm_res(var11_cls.var_vals(),var10_cls.var_vals()) # Sqrt of the sum of the sqs
            Norm_t=max(Norm0[tstep+1],Norm1[tstep+1])
            #DEBUGSTART
            if DEBUG:
                #for var_cls in (var00_cls,var01_cls,var10_cls,var11_cls):
                #    c=check_zero(var_cls.var_vals())
                #    print('before update zero check on',var_cls.name,' is ',c)
                print('solved for tstep',tstep)
            #DEBUGEND
            # Reassign the variables before the next step
            var00_cls.update_var(var01_cls)
            var00_cls.store_tstep(t)
            var10_cls.update_var(var11_cls)
            var01_cls.store_tstep(t)

    vars_cls=(var01_cls,var11_cls)
    return vars_cls, Norm0, Norm1

In [ ]:
def two_step_loop(vars_cls,consts,tvec,test=1,ResultsFolder=""):
    '''Iterate through the time steps estimating solutions using the two step explicit solver.
    :param vars_cls: The spatial variables for variable i at time step j varij_cls. 
    $(varij_cls| i \in [0,1],j \in [0,1])$
    :param consts: consts for forming the variational functions and the simulation tolerance.
    :param tvec: vector with the time at desired timesteps :math:`tvec=(0,dt,...,nt*dt)`
    :param test: indicator, if 1 then the simulation is run for the Heat Eqn
    
    .. code-block::
        Create 2-step solver `Solve`
        for tstep,t in tvec:
            vari(j+1)=Solve(varij,vari(j-1))
            Norm[tstep]=residual(vari(j+1),varij)
            plot(t,vari(j+1))
            Update_Solver
            Update_vars(vari(j-1)=varij,varij=vari(j+1))
        return vari(j+1) Norm
        
    Returns the variable object with the estimated at the final time step and the residual vector from all the time steps.
    :returns: vari(j+1), Norm_vec,... for all variables i
    '''            
    # Retrieve the variables from the inputs
    var00_cls,var10_cls,var11_cls,var01_cls=vars_cls
    V            =var00_cls.function_space
    r_mesh       =var00_cls.mesh
    var02_cls=spatial_variable(V,r_mesh,var00_cls.name+'2')
    var02_cls.set_char_key(var00_cls.char_key)
    var12_cls=spatial_variable(V,r_mesh,var10_cls.name+'2')
    var12_cls.set_char_key(var10_cls.char_key)
    Norm0=np.zeros(len(tvec))
    Norm1=np.zeros(len(tvec))
    Norm_t=1
    tol=consts[0]
    simsteps=2
    
    v0=ufl.TestFunction(V)
    v1=ufl.TestFunction(V)
    tmpv0=ufl.TrialFunction(V)
    tmpv1=ufl.TrialFunction(V)

    #eta_xdmf.write_function(eta2_cls.var,dt)
    #the_xdmf.write_function(the2_cls.var,dt)
    
    
    # Calculate change between timesteps
    Norm0[0]=norm_res(var00_cls.var_vals(),var01_cls.var_vals()) # Sqrt of the sum of the squares.
    Norm1[0]=norm_res(var10_cls.var_vals(),var11_cls.var_vals()) # Sqrt of the sum of the squares
    Norm_t=max(Norm0[0],Norm1[0])


    # Improve solver for two step
    # linear form - inner must be used instead of * for the solvers to work
    L0=linear_form_func(var00_cls.char_key,simsteps,(var00_cls,var01_cls),var10_cls,v0,consts)
    L1=linear_form_func(var010_cls.char_key,simsteps,(var10_cls,var11_cls),var00_cls,v1,consts)
    # bilinear form
    a0=bilinear_form_func(var00_cls.char_key,tmpv0,v0,consts,simsteps,r_mesh,test=0)
    a1=bilinear_form_func(var10_cls.char_key,tmpv1,v1,consts,simsteps,r_mesh,test=0)


    # Create new solvers using the two step variational form
    Solver_2step_0,bvec_0,bc_0=create_solver(a0,L0,r_mesh,V)
    Solver_2step_1,bvec_1,bc_1=create_solver(a1,L1,r_mesh,V)

    # CTIME STEPPING COMPUTATIONS------------------------------------------------------

    # Begin Iterative Simulations --------------------------------------
    check_out=False
    dt=tvec[1]-tvec[0]
    for tstep,t in enumerate(tvec):
        if check_out: # When the residual is reached a steady state has been reached.
            print('Nans in output ')
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        elif Norm_t <=tol: # When the residual is reached a steady state has been reached.
            print('tolerance met %.6f'%Norm_t)
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        elif tstep+2>=len(tvec): 
            print('Time steps exceeded maximum')
            print('Final time %.3f'%(t-dt))
            Norm0=Norm0[1:tstep]
            Norm1=Norm1[1:tstep]
            tvec=tvec[2:tstep+2]
            break
        else:
            #print('Solving for time step',t)
            Solver_2step_0.solve(bvec_0,var02_cls.var.vector)
            Solver_2step_1.solve(bvec_1,var12_cls.var.vector)
            var02_cls.var.x.scatter_forward()
            var12_cls.var.x.scatter_forward()
            # Reassign the variables before the next step
            var00_cls.update_var(var01_cls)
            var10_cls.update_var(var11_cls)
            var01_cls.update_var(var02_cls)
            var11_cls.update_var(var12_cls)
            plot_function(t+dt,var00_cls,tstep,ResultsFolder)
            plot_function(t+dt,var10_cls,tstep,ResultsFolder)
        
            # Write the solution to the xdmf file
            #eta_xdmf.write_function(eta2_cls.var, dt*(tstep+2))
            #the_xdmf.write_function(the2_cls.var,dt*(tstep+2))
            varlist=(var02_cls.var_vals(),var12_cls.var_vals())
            check_out=check_output(varlist)
        
            # Calculate the residual
            Norm0[tstep+1]=norm_res(var00_cls.var_vals(),var01_cls.var_vals()) # Sqrt of the sum of the sqs
            Norm1[tstep+1]=norm_res(var10_cls.var_vals(),var11_cls.var_vals()) # Sqrt of the sum of the sqs
            Norm_t=max(Norm0[tstep+1],Norm1[tstep+1])
            # Update the right hand side reusing the initial vector
            #DEBUGSTART
            for var_cls in (var00_cls,var01_cls,var02_cls,var10_cls,var11_cls,var12_cls):
                c=check_zero(var_cls.var_vals())
                print('before update zero check on',var_cls.name,' is ',c)
            #DEBUGEND
            # RESET eta2,the2
            var02_cls.reset_var()
            var12_cls.reset_var()
            bvec_0=UpdateSolver(bvec_0,a0,L0,bc_0)
            bvec_1=UpdateSolver(bvec_1,a1,L1,bc_1)
    var00_cls.update_var(var02_cls)
    var10_cls.update_var(var12_cls)
    plot_function(t+2*dt,var00_cls,tstep,ResultsFolder)
    plot_function(t+2*dt,var10_cls,tstep,ResultsFolder)
    vars_cls=(var02_cls,var12_cls)
    return vars_cls, Norm0, Norm1

In [ ]:
def change_owner():
    gid=os.getgid()
    uid=1000
    dirname=os.getcwd()
    os.chown(dirname,uid,gid)
    return

# Main Program
The main program deals with the files, plots and simulations.

In [ ]:
def main(filename,simon=0,test=0,simstep=1):
    '''Runs the simulations and plotting functions.
    Ouputs from the simulation are saved as numpy variables. These are then loaded before plotting.
    :param filename: The name of the file containing the input parameters
    :param simon: 0 or 1, if 1 then run simulations, if 0 then only plotting required. default 0.
    :param test: 0 or 1 if 1 then the simulation will run as heat equation
    :param simstep: 1 or 2 number of steps in spatial solver
    Uses functions :py:func:`Simulations(filename,MeshFolder,ResultsFolder,varnames,test)' and :py:func:`plotting(var1,var2,time,foldername)'
    :returns: nothing
    '''
    ResultsFolder='Plots'
    MeshFolder  ='Mesh'
    if test==1:
        ResultsFolder='./DiffusionOnly/'+ResultsFolder
        MeshFolder   ='./DiffusionOnly/'+MeshFolder
    dirlist=(ResultsFolder,MeshFolder)
    etaname     ='eta'
    thetaname   ='theta'
    tname       ='tvec'
    domname     ='boundaries'
    etaresname  ='eta_res'
    theresname  ='the_res'
    varnames=[etaname,thetaname,tname,domname,etaresname,theresname]
    if simon: eta_cls,the_cls,parstr=Simulations(filename,MeshFolder,ResultsFolder,varnames,test,simstep)
    print('Simulations completed')
    change_owner()
    return

## Final Script

In [ ]:
if __name__=='__main__':
    global DEBUG
    DEBUG=True
    filename='Inputs.csv'
    simon=1
    heateqntest=0
    simstep=1
    main(filename,simon,heateqntest,simstep)
    exit()